In [1]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from open3d.t.geometry import TriangleMesh
from tqdm import tqdm 
import time
from OCC.Core.STEPControl import STEPControl_Reader
from OCC.Core.IFSelect import IFSelect_RetDone
from OCC.Core.TopAbs import TopAbs_EDGE, TopAbs_VERTEX
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopoDS import topods
from OCC.Core.BRep import BRep_Tool
from OCC.Core.BRepAdaptor import BRepAdaptor_Curve
from OCC.Core.GeomAbs import GeomAbs_Circle, GeomAbs_Line

In [ ]:
#Pre-processing
dataname = "/home/chris/Code/PointClouds/data/FLIPscans/MortenPlateTopSuperCleaned500k.ply"
pcd = o3d.io.read_point_cloud(dataname)
pcd_center = pcd.get_center()
pcd.translate(-pcd_center)

#Outlier removal
nn = 16
std_multiplier = 10

filtered_pcd = pcd.remove_statistical_outlier(nn,std_multiplier)
outliers = pcd.select_by_index(filtered_pcd[1], invert = True)
outliers.paint_uniform_color([1,0,0])
filtered_pcd = filtered_pcd[0]

#Downsampling
voxel_size = 0.01
pcd_downsampled = filtered_pcd.voxel_down_sample(voxel_size=voxel_size)
print(f'number of points: {len(pcd_downsampled.points)}')

#Extract normals
nn_distance = np.mean([pcd.compute_nearest_neighbor_distance()])
radius_normals = nn_distance*4

pcd_downsampled.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normals, max_nn=16), fast_normal_computation=True)

pcd_downsampled.paint_uniform_color([0.6,0.6,0.6])

front =  [ 0.0, 0.0, 1.0 ]
lookat = [ 4.155493041143778, 3.3619307090130235, 0.041189902146896884 ]
up =  [ 0.0, 1.0, 0.0 ]
zoom = 0.61999999999999988

pcd = pcd_downsampled

# Multi-order Ransac

max_plane_idx = 1
pt_to_plane_dist = 0.4

segment_models = {}
segments = {}
main_surface_idx = 0
largest_surface_points = 0
rest = pcd

for i in range(max_plane_idx):
    #print(f'Run {i}/{max_plane_idx} started. ', end='')
    colors = plt.get_cmap("tab20")(i)
    segment_models[i], inliers = rest.segment_plane(distance_threshold=pt_to_plane_dist,ransac_n=3,num_iterations=50000)
    segments[i] = rest.select_by_index(inliers)
    if len(segments[i].points) > largest_surface_points:
        largest_surface_points = len(segments[i].points) 
        main_surface_idx = i
    segments[i].paint_uniform_color(list(colors[:3]))
    rest = rest.select_by_index(inliers, invert=True)
    print('Done')

# Gap detection
main_surface_pcd = segments[main_surface_idx]
np.asarray(main_surface_pcd.points)[:, 2] = 0

points = np.asarray(main_surface_pcd.points)
pcd_tree = o3d.geometry.KDTreeFlann(main_surface_pcd)

grid_resolution = 0.1
search_radius = 0.5
neighbor_threshold = 4

min_bound = main_surface_pcd.get_min_bound()
max_bound = main_surface_pcd.get_max_bound() 

x_grid = np.arange(min_bound[0], max_bound[0], grid_resolution)
y_grid = np.arange(min_bound[1], max_bound[1], grid_resolution)
z_value = np.mean([min_bound[2], max_bound[2]])

# Generate all query points at once
xv, yv = np.meshgrid(x_grid, y_grid, indexing='ij')
query_points = np.vstack((xv.ravel(), yv.ravel(), np.full_like(xv.ravel(), z_value))).T

gap_points = []

# Efficient neighbor search
for query_point in tqdm(query_points):
    k, idx, _ = pcd_tree.search_radius_vector_3d(query_point, search_radius)
    if k < neighbor_threshold:
        gap_points.append(query_point)

gap_pcd = o3d.geometry.PointCloud()
gap_pcd.points = o3d.utility.Vector3dVector(gap_points)
gap_pcd.paint_uniform_color([1, 0, 0])

#DBScan clustering

eps = 0.30
min_points = 10

labels = np.array(gap_pcd.cluster_dbscan(eps=eps, min_points=min_points, print_progress=True))
max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")
print(labels[0])
base_cmap = plt.get_cmap("tab20")
color_cycle = [base_cmap(i % 20) for i in range(max_label + 1)]
colors = np.array(color_cycle)[labels]
colors[labels<0] = 0
gap_pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

#Filter small clusters

points = np.asarray(gap_pcd.points)
colors = np.asarray(gap_pcd.colors)  
point_threshold = 45
mask = np.zeros(len(points), dtype=bool)
unique_labels = np.unique(labels)

filtered_pcd = o3d.geometry.PointCloud()
filtered_clusters = []
filtered_labels_unique = []
filtered_labels_vector = []

#Filter and assign
for label in unique_labels:
    if label != -1:
        cluster_indices = np.where(labels == label)[0]
        if len(cluster_indices) >= point_threshold:
            mask[cluster_indices] = True
            cluster_points = points[cluster_indices]
            filtered_labels_unique.append(label)

for label in filtered_labels_unique:
    cluster_indices = np.where(labels == label)[0]
    cluster_points = points[cluster_indices]
    filtered_clusters.append(cluster_points)
    cluster_labels = np.tile(label, (cluster_points.shape[0], 1))
    filtered_labels_vector.append(cluster_labels)

total_points = np.vstack([cluster for cluster in filtered_clusters])
filtered_labels = np.vstack([cluster_labels for cluster_labels in filtered_labels_vector]).squeeze()

#print(total_points.shape)
#print(filtered_labels.shape)

filtered_pcd.points = o3d.utility.Vector3dVector(total_points)

#Recolor the filtered pcd
max_label = filtered_labels.max()
base_cmap = plt.get_cmap("tab20")
color_cycle = [base_cmap(i % 20) for i in range(max_label + 1)]
colors = np.array(color_cycle)[filtered_labels]
filtered_pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

In [ ]:
o3d.visualization.draw_geometries([pcd_downsampled, filtered_pcd], zoom=zoom, front=front, lookat=lookat, up=up)

In [ ]:
class ConcaveHullCluster:
    def __init__(self, cluster, k, quick_mode):
        self.cluster = np.delete(cluster, 2, 1)
        self.direction_vector = np.array([-1, 0])
        self.active_points = np.ones(self.cluster.shape[0], dtype=bool)
        self.grid_points = np.ones(self.cluster.shape[0], dtype=bool)
        self.hull_points_final = np.zeros(self.cluster.shape[0], dtype=bool)
        self.k = k
        self.k_list = [8, 10, 12]
        self.hull_points_indices = []
        self.it = 0
        self.breakloop_it = 30000
        self.grid_size = np.array([5, 5])
        self.collinearity = False
        self.fast = quick_mode
    
    def calculate_distances(self, hull_point, active_indices):
        return np.sqrt(np.sum(np.square(self.cluster[active_indices] - hull_point), axis=1))
    
    def get_knn(self, current_hull_point):
        active_grid_points = np.logical_and(self.active_points, self.grid_points)
        active_grid_points_indices = np.nonzero(active_grid_points)[0]
        distances = self.calculate_distances(current_hull_point, active_grid_points_indices)
        sorted_indices = np.argsort(distances)
        neighbor_indices = [active_grid_points_indices[i] for i in sorted_indices[:self.k]]
        return neighbor_indices, self.cluster[neighbor_indices]
    
    def find_lowest_point(self):
        lowest_y = np.amin(self.cluster[:,1])
        lowest_y_indices = np.where(self.cluster[:, 1] == lowest_y)[0]

        if lowest_y_indices.shape[0] == 1:
            lowest_point_index = lowest_y_indices[0]
            lowest_point = self.cluster[lowest_point_index]
        elif lowest_y_indices.shape[0] > 1:
            lowest_y_points = self.cluster[lowest_y_indices]
            lowest_x = np.amin(lowest_y_points[:, 0])
            lowest_point_index = np.where((self.cluster[:,0] == lowest_x) & (self.cluster[:,1] == lowest_y))[0][0]
            lowest_point = np.array([lowest_x, lowest_y])
        return lowest_point, lowest_point_index
    
    def get_points_inside_grid(self, current_hull_point):
        grid_min = current_hull_point - self.grid_size/2
        grid_max = current_hull_point + self.grid_size/2
        within_grid = np.all((self.cluster >= grid_min) & (self.cluster <= grid_max), axis=1)
        self.grid_points = within_grid

    def unit_vector(self, vector):
        return vector/np.linalg.norm(vector)
    
    def ccw(self,A,B,C):
        return (C[1]-A[1])*(B[0]-A[0]) > (B[1]-A[1])*(C[0]-A[0])

    def check_collinearity(self, A, B, C):
        area = A[0]*(B[1]-C[1]) + B[0]*(C[1]-A[1]) + C[0]*(A[1]-B[1])
        if area==0:
            return True
        return False

    def check_parallelism(self, vector_alpha, vector_beta):
        determinant = vector_alpha[0] * vector_beta[1] - vector_alpha[1] * vector_beta[0]
        return np.isclose(determinant, 0)

    def intersect_with_collinearity(self, A, B, C, D):
        if np.array_equal(A, C) or np.array_equal(A, D) or np.array_equal(B, C) or np.array_equal(B, D):
            return False
        if self.check_collinearity(A, C, D) or self.check_collinearity(B, C, D):
            return False
        return self.ccw(A, C, D) != self.ccw(B, C, D) and self.ccw(A, B, C) != self.ccw(A, B, D)
    
    def intersect_without_collinearity(self, A, B, C, D):
        if np.array_equal(A, C) or np.array_equal(A, D) or np.array_equal(B, C) or np.array_equal(B, D):
            return False
        return self.ccw(A, C, D) != self.ccw(B, C, D) and self.ccw(A, B, C) != self.ccw(A, B, D)
    
    def increase_k(self, current_hull_point, neighbors):
        for z in range(1, len(self.k_list)):
            self.k = self.k_list[z]
            print(f'Increasing k to {self.k}', flush=True)
            _, neighbors = self.get_knn(current_hull_point)
            chosen_neighbor = self.choose_neighbor(current_hull_point, neighbors)
            if chosen_neighbor is None:
                continue
            else:
                self.k = self.k_list[0]
                return chosen_neighbor
        self.k = self.k_list[0]

    def enable_collinearity(self, current_hull_point, neighbors):
        print('Enabling collinearity')
        self.collinearity = True
        for z in range(len(self.k_list)):
            self.k = self.k_list[z]
            print(f'Trying with k: {self.k}', flush=True)
            _, neighbors = self.get_knn(current_hull_point)
            chosen_neighbor = self.choose_neighbor(current_hull_point, neighbors)
            if chosen_neighbor is None:
                continue
            else:
                self.k = self.k_list[0]
                self.collinearity = False
                return chosen_neighbor
        self.k = self.k_list[0]
        self.collinearity = False

    def find_all_indices(self, target_list, query_value):
        return [i for i, x in enumerate(target_list) if x == query_value]
    
    def are_consecutive(self, index1, index2):
        return abs(index1 - index2) == 1
    
    def choose_neighbor(self, current_hull_point, neighbors):
        vectors = neighbors - current_hull_point
        dot_products = np.dot(vectors,self.direction_vector)
        cross_products = np.cross(self.direction_vector, vectors)
        norms = np.linalg.norm(vectors, axis=1)
        cos_angles = np.clip(dot_products/norms, -1.0, 1.0)
        angles = np.arccos(cos_angles)

        right_turn_indices= np.where(cross_products <= 0)[0]
        left_turn_indices= np.where(cross_products > 0)[0]
        
        right_turn_angles = angles[right_turn_indices]
        left_turn_angles = angles[left_turn_indices]

        sorted_right_turn_indices = right_turn_indices[np.argsort(-right_turn_angles)]
        sorted_left_turn_indices = left_turn_indices[np.argsort(left_turn_angles)]

        combined_indices = np.concatenate([sorted_right_turn_indices, sorted_left_turn_indices])

        if self.fast:
            grid_hull_points_indices = [idx for idx in self.hull_points_indices if self.grid_points[idx]]

            if len(grid_hull_points_indices) > 25:
                grid_hull_points_indices = grid_hull_points_indices[-25:]
            
            for index in combined_indices:
                chosen_neighbor = neighbors[index]
                C = current_hull_point
                D = chosen_neighbor
                intersects = False

                for i in range(1, len(grid_hull_points_indices)):
                    A = self.cluster[grid_hull_points_indices[i-1]]
                    B = self.cluster[grid_hull_points_indices[i]]
                    vector_alpha = B-A
                    vector_beta = D-C
                    if self.collinearity == False:
                        if self.intersect_without_collinearity(A, B, C, D) and not self.check_parallelism(vector_alpha, vector_beta):
                            #print('Intersection!')
                            intersects = True
                            break
                    else:
                        if self.intersect_with_collinearity(A, B, C, D) and not self.check_parallelism(vector_alpha, vector_beta):
                            #print('Intersection!')
                            intersects = True
                            break
            
                if not intersects:
                    new_direction_vector = self.unit_vector(chosen_neighbor - current_hull_point)
                    self.direction_vector = new_direction_vector
                    #self.hull_points.append(chosen_neighbor)
                    return chosen_neighbor
            #self.k = int(2*self.k)
            return None
        
        else:
            grid_hull_points_indices = [idx for idx in self.hull_points_indices if self.grid_points[idx]]

            for index in combined_indices:
                chosen_neighbor = neighbors[index]
                C = current_hull_point
                D = chosen_neighbor
                intersects = False

                for i in range(1, len(grid_hull_points_indices)):
                    A_index = grid_hull_points_indices[i - 1]
                    B_index = grid_hull_points_indices[i]

                    A_indices = self.find_all_indices(self.hull_points_indices, A_index)
                    B_indices = self.find_all_indices(self.hull_points_indices, B_index)

                    is_consecutive = any(self.are_consecutive(A_idx, B_idx) for A_idx in A_indices for B_idx in B_indices)

                    if is_consecutive:
                        A = self.cluster[A_index]
                        B = self.cluster[B_index]
                        vector_alpha = B - A
                        vector_beta = D - C
                        if self.collinearity == False:
                            if self.intersect_without_collinearity(A, B, C, D) and not self.check_parallelism(vector_alpha, vector_beta):
                                intersects = True
                                break
                        else:
                            if self.intersect_with_collinearity(A, B, C, D) and not self.check_parallelism(vector_alpha, vector_beta):
                                intersects = True
                                break

                if not intersects:
                    new_direction_vector = self.unit_vector(chosen_neighbor - current_hull_point)
                    self.direction_vector = new_direction_vector
                    return chosen_neighbor

            return None

    def indices_to_array(self):
        self.hull_points_final[self.hull_points_indices] = True
        return self.hull_points_final
    
    def compute_concave_hull(self):
        self.lowest_point, self.lowest_point_index = self.find_lowest_point()
        self.hull_points_indices.append(self.lowest_point_index)
        self.active_points[self.lowest_point_index] = False
        self.it += 1
        while True:
            print(f'Iteration: {self.it}', end='\r')
            current_hull_point_index = self.hull_points_indices[-1]
            current_hull_point = self.cluster[current_hull_point_index]
            self.active_points[current_hull_point_index] = False
            if self.it == self.breakloop_it:
                print('Time to stop this madness', flush=True)
                return self.indices_to_array()
            self.get_points_inside_grid(current_hull_point)
            _, neighbors = self.get_knn(current_hull_point)
            chosen_neighbor = self.choose_neighbor(current_hull_point, neighbors)
            if chosen_neighbor is None:
                chosen_neighbor = self.increase_k(current_hull_point, neighbors)
            if chosen_neighbor is None:
                chosen_neighbor = self.enable_collinearity(current_hull_point, neighbors)
            if chosen_neighbor is None:
                print('Couldnt find neighbor or find lowest point, closing hull.')
                return self.indices_to_array()
            chosen_neighbor_index = np.where((self.cluster[:,0] == chosen_neighbor[0]) & (self.cluster[:,1] == chosen_neighbor[1]))[0][0]
            if chosen_neighbor_index == self.lowest_point_index:
                print('\nSuccess!\n',flush=True)
                return self.indices_to_array()

            self.hull_points_indices.append(chosen_neighbor_index)
            if self.it >= int(2*self.k):
                past_index = self.hull_points_indices[self.it - int(2*self.k)]
                self.active_points[past_index] = True
            self.it +=1

In [ ]:
hull_dictionary = {}
for i, cluster_points in enumerate(filtered_clusters):
    print(f'Cluster {i+1}/{len(filtered_clusters)}')
    print(25*'-')
    #print(f'Number of points in the cluster: {cluster_points.shape[0]}')
    cluster = ConcaveHullCluster(cluster_points, k=8, quick_mode=True)
    hull_dictionary[i] = cluster.compute_concave_hull()

In [ ]:
#Visualize Hulls

hull_list = []

for i in range(len(filtered_clusters)):
    cluster_points = filtered_clusters[i]
    hull_points = cluster_points[hull_dictionary[i]]
    hull_list.append(hull_points)

total_points = np.vstack([cluster for cluster in filtered_clusters])
filtered_labels = np.vstack([cluster_labels for cluster_labels in filtered_labels_vector]).squeeze()
total_hull_points = np.vstack([hull_points for hull_points in hull_list])
total_hull_colors = np.tile((1,0.65,0.05), (total_hull_points.shape[0], 1))
#print(total_points.shape)
#print(filtered_labels.shape)

filtered_pcd.points = o3d.utility.Vector3dVector(total_points)

hull_pcd = o3d.geometry.PointCloud()
hull_pcd.points = o3d.utility.Vector3dVector(total_hull_points)
hull_pcd.colors = o3d.utility.Vector3dVector(total_hull_colors)
#Recolor the filtered pcd
max_label = filtered_labels.max()
base_cmap = plt.get_cmap("tab20")
color_cycle = [base_cmap(i % 20) for i in range(max_label + 1)]
colors = np.array(color_cycle)[filtered_labels]
filtered_pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

o3d.visualization.draw_geometries([filtered_pcd, hull_pcd], zoom=zoom, front=front, lookat=lookat, up=up)

In [ ]:
from scipy.spatial import KDTree

#Transfer hulls over to the point cloud

# Convert the main surface point cloud to a numpy array
main_surface_points = np.asarray(main_surface_pcd.points)

# Build a KDTree for the main surface points
kd_tree = KDTree(main_surface_points)

# Initialize a dictionary to store the mapped hull points on the main surface
mapped_hull_dictionary = {}

for i in range(len(filtered_clusters)):
    cluster_points = filtered_clusters[i]
    hull_points = cluster_points[hull_dictionary[i]]
    
    mapped_hull_indices = []
    for hull_point in hull_points:
        # Find the index of the closest point in the original point cloud
        _, idx = kd_tree.query(hull_point)
        mapped_hull_indices.append(idx)
    
    # Store the mapped hull indices in the dictionary
    mapped_hull_dictionary[i] = mapped_hull_indices

# To visualize the hull points on the original point cloud
mapped_hull_pcd = o3d.geometry.PointCloud()
for idx_list in mapped_hull_dictionary.values():
    mapped_hull_pcd.points.extend(main_surface_points[idx_list])
mapped_hull_pcd.paint_uniform_color([0.1, 1, 0.1])

# Visualize the result
o3d.visualization.draw_geometries([mapped_hull_pcd])

In [ ]:
# Load the STEP file
def load_step_file(file_path):
    step_reader = STEPControl_Reader()
    status = step_reader.ReadFile(file_path)
    if status == IFSelect_RetDone:
        step_reader.TransferRoots()
        shape = step_reader.OneShape()
        return shape
    else:
        raise Exception("Error: Cannot read STEP file.")

# Extract edges and categorize them
def extract_features(shape):
    circles = []
    lines = []
    exp_edge = TopExp_Explorer(shape, TopAbs_EDGE)
    while exp_edge.More():
        edge = topods.Edge(exp_edge.Current())
        curve_adaptor = BRepAdaptor_Curve(edge)
        curve_type = curve_adaptor.GetType()
        
        if curve_type == GeomAbs_Circle:
            circ = curve_adaptor.Circle()
            center = circ.Location()
            radius = circ.Radius()
            circles.append({
                'type': 'circle',
                'center': (center.X(), center.Y(), center.Z()),
                'radius': radius
            })
        elif curve_type == GeomAbs_Line:
            # Get the start and end points of the line
            exp_vertex = TopExp_Explorer(edge, TopAbs_VERTEX)
            vertices = []
            while exp_vertex.More():
                vertex = topods.Vertex(exp_vertex.Current())
                vertices.append(BRep_Tool.Pnt(vertex))
                exp_vertex.Next()
            if len(vertices) == 2:
                start_point = vertices[0]
                end_point = vertices[1]
                lines.append({
                    'type': 'line',
                    'start': (start_point.X(), start_point.Y(), start_point.Z()),
                    'end': (end_point.X(), end_point.Y(), end_point.Z())
                })
        exp_edge.Next()
    
    print(f"Extracted {len(circles)} circles and {len(lines)} lines")
    return circles, lines

# Calculate distance between two points
def distance(point1, point2):
    return np.linalg.norm(np.array(point1) - np.array(point2))

# Group features to form composite shapes
def group_features(circles, lines, tol_distance=1.0):
    composite_shapes = []
    
    # Example logic: Identify squares with rounded corners
    for line in lines:
        line_start = line['start']
        line_end = line['end']
        line_length = distance(line_start, line_end)
        
        matching_circles = []
        for circle in circles:
            circle_center = circle['center']
            circle_radius = circle['radius']
            
            # Check if the circle center is close to either end of the line
            if (distance(circle_center, line_start) < tol_distance + circle_radius or
                distance(circle_center, line_end) < tol_distance + circle_radius):
                matching_circles.append(circle)
        
        # Assume a composite shape if there are exactly 4 matching circles
        if len(matching_circles) == 4:
            composite_shapes.append({
                'type': 'rounded_square',
                'line': line,
                'circles': matching_circles
            })
    
    print(f"Identified {len(composite_shapes)} composite shapes")
    return composite_shapes

# Function to create a flat circle (disk) in Open3D
def create_flat_circle(radius, center, resolution=30, height=0.01):
    mesh = o3d.geometry.TriangleMesh.create_cylinder(radius=radius, height=height, resolution=resolution)
    mesh.translate(center - np.array([0, 0, height / 2]))  # Adjust the center to be flat on the XY plane
    return mesh

# Visualize features using Open3D
def visualize_features(circles, lines, composite_shapes):
    geometries = []
    mapped_hull_points = np.asarray(mapped_hull_pcd.points)
    mapped_hull_points[:, 0] += 60
    mapped_hull_points[:, 1] -= 50
    mapped_hull_pcd.points = o3d.utility.Vector3dVector(mapped_hull_points)
    geometries.append(mapped_hull_pcd)

    # Add circles to the visualization
    for circle in circles:
        center = np.array(circle['center'])
        radius = circle['radius']
        circle_mesh = create_flat_circle(radius=radius, center=center)
        circle_mesh.paint_uniform_color([1, 0, 0])  # Red color for circles
        geometries.append(circle_mesh)

    # Add lines to the visualization
    for line in lines:
        start = np.array(line['start'])
        end = np.array(line['end'])
        line_set = o3d.geometry.LineSet()
        line_set.points = o3d.utility.Vector3dVector([start, end])
        line_set.lines = o3d.utility.Vector2iVector([[0, 1]])
        line_set.paint_uniform_color([0, 1, 0])  # Green color for lines
        geometries.append(line_set)

    # Add composite shapes to the visualization
    for shape in composite_shapes:
        for circle in shape['circles']:
            center = np.array(circle['center'])
            radius = circle['radius']
            circle_mesh = create_flat_circle(radius=radius, center=center)
            circle_mesh.paint_uniform_color([0, 0, 1])  # Blue color for composite shape circles
            geometries.append(circle_mesh)
        
        line = shape['line']
        start = np.array(line['start'])
        end = np.array(line['end'])
        line_set = o3d.geometry.LineSet()
        line_set.points = o3d.utility.Vector3dVector([start, end])
        line_set.lines = o3d.utility.Vector2iVector([[0, 1]])
        line_set.paint_uniform_color([1, 1, 0])  # Yellow color for composite shape lines
        geometries.append(line_set)

    # Visualize all geometries
    o3d.visualization.draw_geometries(geometries)

# Main execution
if __name__ == "__main__":
    step_file_path = step_file_path = "/home/chris/Code/PointClouds/data/other_files/MortenPartSTEPVersion.STEP"  # Replace with your actual file path
    shape = load_step_file(step_file_path)
    
    circles, lines = extract_features(shape)
    composite_shapes = group_features(circles, lines, tol_distance=10.0)  # Adjust tol_distance as needed
    
    # Visualize the extracted features and composite shapes
    visualize_features(circles, lines, composite_shapes)